In [ ]:
import openai
import pandas as pd
import random
import time
import os
import json
from collections import Counter


# Set up OpenAI API key
OPENAI_API_KEY = "your-api-key-here"
MODEL_NAME = "your-fine-tuned-model-name"  # Update with your fine-tuned model name
openai.api_key = OPENAI_API_KEY

# Load the train dataset as RAG source
train_dataset_path = "data/train_dataset.json"
with open(train_dataset_path, "r", encoding='utf-8') as f:
    rag_data = json.load(f)

# Convert to DataFrame
rag_df = pd.DataFrame(rag_data)

# Ensure diverse sampling across Grade, Topic, and Sub-topic
def get_balanced_samples(df, num_samples=1000):
    grouped = df.groupby(["Grade", "Topic", "Sub Topic"])
    selected_samples = []

    while len(selected_samples) < num_samples:
        for _, group in grouped:
            if len(selected_samples) >= num_samples:
                break
            sampled_row = group.sample(1)
            selected_samples.append(sampled_row)

    return pd.concat(selected_samples).reset_index(drop=True)

balanced_rag_df = get_balanced_samples(rag_df, 1000)

# Define function to retrieve relevant context from RAG dataset
def retrieve_relevant_context(grade, topic, sub_topic):
    relevant_data = rag_df[(rag_df["Grade"] == grade) & (rag_df["Topic"] == topic) & (rag_df["Sub Topic"] == sub_topic)]
    if not relevant_data.empty:
        return relevant_data.sample(1).to_dict(orient='records')[0]["Problem"]
    return "No relevant example found."

# Generate student responses dynamically
def generate_student_response(grade, topic, sub_topic):
    prompts = [
        f"A student in grade {grade} is struggling with {sub_topic} in {topic}. Generate a student response.",
        f"What is a typical way a student might respond to {sub_topic} in {topic}?",
        f"Simulate a student attempting {sub_topic} in {topic} and making a common mistake."
    ]
    return random.choice(prompts)

# Call the fine-tuned model with RAG context to generate student responses
def get_student_response(prompt, context):
    try:
        response = openai.ChatCompletion.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": "You are a student responding to a math problem."},
                {"role": "user", "content": f"Context: {context}\n\n{prompt}"}
            ],
            max_tokens=200
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print("Error generating response:", e)
        return None

# Generate dataset
student_responses = []

for _, row in balanced_rag_df.iterrows():
    grade = row["Grade"]
    topic = row["Topic"]
    sub_topic = row["Sub Topic"]

    student_prompt = generate_student_response(grade, topic, sub_topic)
    context = retrieve_relevant_context(grade, topic, sub_topic)
    student_reply = get_student_response(student_prompt, context)

    if student_reply:
        student_responses.append([grade, topic, sub_topic, student_prompt, context, student_reply])

    # Avoid rate limits
    time.sleep(1)

# Convert to DataFrame
student_responses_df = pd.DataFrame(student_responses, columns=["Grade", "Topic", "Sub-topic", "Student Prompt", "RAG Context", "Student Response"])

# Save dataset
student_responses_df.to_csv("generated_student_responses_rag.csv", index=False)
print("Dataset generation complete. Saved as generated_student_responses_rag.csv")


In [ ]:
# Install OpenAI library (if not installed)
!pip install openai

# Import necessary libraries
import openai
import json
import pandas as pd
import time
from openai import OpenAI

# Set OpenAI API key (Replace with your key)
secret_key = ''
organization_id = ''
openai.organization = organization_id
openai.api_key = secret_key
os.environ["OPENAI_API_KEY"] = secret_key
client = OpenAI(organization=organization_id)

# Define file paths
train_data_path = "data/train_dataset.json"
formatted_data_path = "data/student_train_dataset.jsonl"

#Uncomment following code to generate fine-tuned model, once model generated, then comment this code and execute further
"""
# Load training dataset
with open(train_data_path, "r", encoding="utf-8") as f:
    train_data = json.load(f)

# Convert dataset to OpenAI fine-tuning format
formatted_data = []

for row in train_data:
    prompt = f"Grade: {row.get('Grade', '')}, Topic: {row.get('Topic', '')}, Sub-topic: {row.get('Sub Topic', '')}"
    student_mistake = row.get('Student\'s Mistake Prompt', '')
    completion = f"#$?#Challenge Type: {row.get('Challenge Type', '')}#$?#Problem: {row.get('Problem', '')}#$?#Student's Mistake: {student_mistake}#$?#"

    if prompt and completion:
        formatted_data.append({
            "messages": [
                {"role": "system", "content": "You are a student making mistakes while solving math problems."},
                {"role": "user", "content": prompt},
                {"role": "assistant", "content": completion}
            ]
        })

# Save formatted data as JSONL
with open(formatted_data_path, "w", encoding="utf-8") as f:
    for entry in formatted_data:
        json.dump(entry, f)
        f.write("\n")

print(f" Formatted dataset saved at: {formatted_data_path}")

# Upload dataset to OpenAI for fine-tuning
response = client.files.create(
    file=open(formatted_data_path, "rb"),
    purpose="fine-tune"
)
print(response)

file_id = "file-QVGgcXbLtwkFDsYHQqH8hB"#response.id
print(f" Dataset uploaded to OpenAI. File ID: {file_id}")

# Start fine-tuning job
fine_tune_response = client.fine_tuning.jobs.create(
    training_file=file_id,
    model="gpt-3.5-turbo"  # Change to "gpt-4-turbo" if needed
)
print(fine_tune_response)
job_id = fine_tune_response.id
print(f"Fine-tuning job started. Job ID: {job_id}")
"""
job_id = "your fine-tuned job id"
# Monitor fine-tuning progress
while True:
    status = client.fine_tuning.jobs.retrieve(job_id)
    status_state = status.status

    print(f"⏳ Fine-tuning status: {status_state}")

    if status_state in ["succeeded", "failed", "cancelled"]:
        print(f" Fine-tuning completed with status: {status_state}")
        break

    time.sleep(60)  # Wait for a minute before checking again

# Get the fine-tuned model name
if status_state == "succeeded":
    fine_tuned_model = status.fine_tuned_model
    print(f"🎉 Fine-tuned student model is ready: {fine_tuned_model}")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
⏳ Fine-tuning status: validating_files
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status: running
⏳ Fine-tuning status

In [ ]:
print(status)

FineTuningJob(id='ftjob-Y91humBueMVUpPUlFFrJgVkc', created_at=1739934029, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::B2VFnwd2', finished_at=1739936493, hyperparameters=Hyperparameters(batch_size=7, learning_rate_multiplier=2.0, n_epochs=3), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-KaLJlf6HI91T8c0b7uy8XUt2', result_files=['file-NuHhJ4idMwRfPBGhBMrLMC'], seed=1626645714, status='succeeded', trained_tokens=838488, training_file='file-QVGgcXbLtwkFDsYHQqH8hB', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=7, learning_rate_multiplier=2.0, n_epochs=3)), type='supervised'), user_provided_suffix=None)


In [ ]:
# Install OpenAI library (if not installed)
!pip install openai pandas

# Import necessary libraries
import openai
import pandas as pd
import random
import time
import json
import os
from openai import OpenAI

FINE_TUNED_MODEL = ""  # Update with your fine-tuned model name

# Define file paths
selected_combinations_path = "data/50_GTS_Combination.csv"
output_csv_path = "data/student_fine_tuned_results.csv"

# Load selected 50 (Grade, Topic, Sub-topic) combinations
selected_combinations = pd.read_csv(selected_combinations_path)

# Function to generate student mistake prompts
def generate_student_mistake(grade, topic, sub_topic):
    prompts = [
        f"A grade {grade} student is struggling with {sub_topic} in {topic}. Generate a common mistake and the problem statement.",
        f"What is a typical error students make in {sub_topic}? Provide a problem where such a mistake happens.",
        f"Create a math problem related to {sub_topic} in {topic} where students usually make errors."
    ]
    start = random.choice(prompts)
    output_format = (
        "Output should be only a valid JSON with the following format: "
        "{\"challenge_type\":\"<challenge type>\", \"question\":\"<math problem>\", \"mistake\":\"<student's mistake>\"}"
    )

    return start + " " + output_format

# Function to call OpenAI fine-tuned student model
def get_math_problem(prompt):
    try:
        response = client.chat.completions.create(
            model=FINE_TUNED_MODEL,
            messages=[
                {"role": "system", "content": "You are a student making mistakes while solving math problems."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500
        )

        output = response.choices[0].message.content.strip()
        output = output.replace("```json", "").replace("```", "").replace("\\", "")
        if "#$?#" in output:
            output_l = output.split("#$?#")
            challenge_type = output_l[1].split("Challenge Type:")[1]
            question = output_l[2].split("Problem:")[1]
            mistake = output_l[3].split("Student's Mistake:")[1]
        else:
            json_o = json.loads(output)

        return json_o["challenge_type"], json_o["question"], json_o["mistake"]

    except Exception as e:
        print(" Error generating response:", e)
        return None

# Generate dataset
math_data = []

for _, row in selected_combinations.iterrows():
    grade, topic, sub_topic = row["Grade"], row["Topic"], row["Sub Topic"]

    num_samples = 100
    for _ in range(num_samples):
        student_prompt = generate_student_mistake(grade, topic, sub_topic)
        print(f" Generating sample for: {grade}, {topic}, {sub_topic}")

        response = get_math_problem(student_prompt)

        if response:
            print(response)
            challenge_type, question, mistake = response
            math_data.append([grade, topic, sub_topic, challenge_type, question, mistake])

        # Avoid rate limits
        time.sleep(1)

# Convert to DataFrame
math_df = pd.DataFrame(math_data, columns=["Grade", "Topic", "Sub-topic", "Challenge Type", "Question", "Student Mistake"])

# Save dataset to Google Drive
math_df.to_csv(output_csv_path, index=False)

print(f" Dataset generation complete. Saved at: {output_csv_path}")


Streaming output truncated to the last 5000 lines.
('Procedural Fraction Error', 'How long is 13 1/4 inches on a ruler with 16 marks per inch?', '- Adding improperly to convert to a single fraction')
📝 Generating sample for: K3, Measurement, measuring lengths (including in fractions of an inch)
('Procedural', 'Convert 1 7/8 inches to an improper fraction of an inch.', 'Using mixed numbers instead of decimals for ease of calculation.')
📝 Generating sample for: K3, Measurement, measuring lengths (including in fractions of an inch)
('Procedural Error', 'Measure the length to the nearest 1/4 inch.', 'Reading the tape measure at 1/8 inch increments.')
📝 Generating sample for: K3, Measurement, measuring lengths (including in fractions of an inch)
('Procedural', 'Measure 4 1/4 inches and mark the line.', 'Interpreting the measurement as 4.14 instead of 4.25 inches.')
📝 Generating sample for: K3, Measurement, measuring lengths (including in fractions of an inch)
('Procedural Error', 'Measure 2